In [1]:
import pymysql
import yaml
import argparse
import getpass
from preprossesor import Preporcessor, dataload, train_folder_dir, CreatePath,seed_everything
from train import TestModel,TrainModel
import os
import pandas as pd
import numpy as np
import torch
from postprosseor import AnomalyGraphCapture 
from z_score import ZscoreCalc
import matplotlib.pyplot as plt
import pickle
import csv
import sys
import datetime
from multiprocessing import Pool
from DataManager import PathCreator,LoadThreshold,SaveThreshold,Thresholding, RulePathCreator ,GetData, GetLastTime,GetTrainData,GetInferenceData,GetColumn,InferenceDirCreator, TrainDirCreator
import gc
###### 0508 버전 
## Nan 값이 있는 것은 데이터가 빈케이스이므로  52번 line 에서 예외 처리함
## col 값 만 feature로 활용 -> 나머지 추가하는가 결국 별로 partial_scaler = False , 후처리때 고민하는게 좋아 보임, No Day to num/  No Diff
## Random Seed 고정 
## 실시간으로 재학습 할꺼임 -> 45일치 학습했는데 초반에 데이터가 비어짐
## AnomalyScoreCapture는 
def train(args, train_list):
    seq_len = args['seq_len']
    n_split = args['CV']
    device =  args['GPU']
    lastmin = 60*24*30
    
    for db,table,port,col in train_list:
        #### Seed 
        seed_everything(42)
        model_list = ["MCD_LSTM", "DCdetector", "TimesNet","TranAD","AnomalyTransformer","Rule_Model"]
        model_type = model_list[0]

        table_path = table.replace("|","_")
        data_path  = os.path.join(db,table_path)
        port_path = port.replace("/","_")
        col_path=col.replace("/","_")
        feature_list = col
        
        # lastdate = datetime.datetime(2024,5,10)# 
        lastdate = GetLastTime(db=db,table= table)
       # print(os.path.exists(model_path) , os. path.exists(scaler_path) , 

        
        model_path, scaler_path ,z_path, df_path, Th_path, save_path_pkl,save_path_csv = PathCreator(args,data_path,model_type,port_path,col_path,lastdate,db,table_path)
        if not (args["FT"]) :
            print("Fine Tuning is on False")
            if os.path.exists(model_path) and os. path.exists(scaler_path) and os.path.exists(Th_path):
                print("있어용")
                print("지나가요~~~~~~")
                continue 
                continue 
        
        train_df, _= GetTrainData(db=db,table=table,col=col,port=port,lastmin= lastmin)
        print("train date time",train_df.shape)
        if train_df.shape[0] < 10 :#: 학습시 nan의 주범
            print("없어용")
            print("지나가요~~~~~~")
            continue
            

       
        prep = Preporcessor(train_df)
        
        yaml_path = './test_data/model/configs/configs.yaml'


        model_list = ["MCD_LSTM", "DCdetector", "TimesNet","TranAD","AnomalyTransformer","Rule_Model"]
        model_list = [model_list[-2]]
        
        table_path = table.replace("|","_")
        data_path  = os.path.join(db,table_path)


        with open(yaml_path) as f:
            d = yaml.safe_load(f)
            try:
                model_configs = d[model_type] #args.model
            except KeyError:
                print(f'config file does not contain default parameter settings of {model_type} ') #{args.model}
                model_configs = {}
        
        #### 저장 경로
        TrainDirCreator(args, data_path, model_list,lastdate)
        
        prep.PortSelection(port, lastdate=lastdate, lasttime=lastmin, padding =True) ### PADDING 제거 
        prep.DayToNum() 

        port_path = port.replace("/","_")
        col_path=col.replace("/","_")
        
        
        model_path, scaler_path ,z_path, df_path, Th_path, save_path_pkl,save_path_csv = PathCreator(args,data_path,model_type,port_path,col_path,lastdate,db,table_path)

        prep.Scaler( col=col, scaler_type = "Standard", path= scaler_path, save=True)
        # prep.Diff(col=col, n_diff=1)
        # print("df",prep.port_df)
#         feature_list = [col,'DAY_OF_WEEK', 'TIME_OF_DAY','col_1_diff']
        feature_list = col # [col,'col_1_diff']              
                        
                        
############# 모델 불러오기 및 재학습 ###########
        model_w=TrainModel(model_configs = model_configs,
            model_type = model_type, 
            n_splits=n_split,
            data = prep ,
            col=col, 
            device= device, #"ms",
            CV=True, 
            epochs=30,
            seq_len =seq_len,
            feature_list=feature_list,
            fine_tuning =False,
            model_path = model_path,
            partial_scaling=False)

        torch.save(model_w, model_path)
        # print(torch.load(model_path))
        print("Save Model-weights in :",model_path)

        gc.collect()
        torch.cuda.empty_cache()
        
        _, test_df = TestModel(model_configs = model_configs, 
            model_type = model_type, 
            model_path = model_path,
            n_splits=n_split,
            data = prep ,col=col, 
            device=device,
            CV=True, 
            epochs=10,
            seq_len =seq_len,
            feature_list=feature_list,
            partial_scaling=False)    

        x =test_df["SCORE"][args["seq_len"]-1:]
        test_df=pd.DataFrame(test_df)
        print("testset Score :",x)
        threshold = SaveThreshold(x,0.02,Th_path) +0.0000001
        print(" Threshold :", threshold)
        print("Save TH in :",Th_path)
        threshold = LoadThreshold(Th_path)


# ######### 학습 결과 저장 ########################
        prep.InverseScaler(col, path=scaler_path)
        test_df["VALUE"]=prep.port_df[f"{col}_ORIGIN"]
        test_df[f"{col}'s Abnormality"] = False
        test_df[f"{col}'s Threshold"] = threshold
        test_df.loc[ test_df["SCORE"] >= threshold,f"{col}'s Abnormality"] = True    
        test_df.to_csv(df_path,index=False)
        AnomalyGraphCapture(test_df[6912:],col,"SCORE",df_path,seq_len,"AS",threshold)
        print("Save trained Score in", df_path)  
#################################################
def process_train(args, train_item):
    db, table, port, col = train_item
    # train 함수 호출
    train(args, [(db, table, port, col)])
##--------------------------------------------------------------------------------------------------------------------

args = {"server":'server1', "mp_processor":1,"model_path" : '/mnt/lv02/', 'checker_path' : './trainable_checker_v4.1.csv', 'save_path' : '/mnt/lv02/save_path/'
       , 'GPU':'cuda:1', 'DB_set':5,'seq_len':8,'CV':1}

##--------------------------------------------------------------------------------------------------------------------     
trainable_csv=pd.read_csv("./trainable_checker_v4.1.csv")
trainable_csv = trainable_csv.loc[trainable_csv['AT_model'] ==True,:]

test_condition1 =((trainable_csv['DB_name'] =="GR_AMF21_5M")|(trainable_csv['DB_name'] =="GR_SMF01_5M"))
# |(trainable_csv['DB_name'] =="GR_UPF01_5M") )

#     # test_condition2 =((trainable_csv['table_name'] =="SMDNN|SE") | (trainable_csv['table_name'] =="PFMC|PFCP")| (trainable_csv['table_name'] =="COSVC|SMPOL"))
#     test_condition3 = ((trainable_csv['col_name'] =="AVG_CONNECTED(count)") )
#     # test_condition4 = ((trainable_csv['port'] =="DNN100-internet_SST1_SDNONE") | (trainable_csv['port'] =="PEER39.18.166.1 (BGJU_UPF01_SS)")| (trainable_csv['port'] =="PEER10.124.34.6"))

test_index  = test_condition1  #& test_condition3 & test_condition3 # & test_condition4
#     print("train list length is : ",np.sum(test_index) ) 

trainable_csv = trainable_csv.loc[test_index,:]   

trainable_csv = trainable_csv.loc[trainable_csv['col_name'] !="DATETIME",:]
trainable_csv = trainable_csv.loc[trainable_csv['table_name'] !='PAG|SM2',:]
trainable_csv = trainable_csv.loc[trainable_csv['table_name'] !='PAG|SM1',:]


trainable_csv_tmp1 = trainable_csv.set_index(["DB_name", "table_name", "port", "col_name"])

nf = { "SET_1": ['BS_AMF22_5M','BS_SMF02_5M','BS_SMF03_5M','BS_UPF01_5M','BS_UPF03_5M','DJ_UPF03_5M','DJ_AMF21_5M','DJ_AMF22_5M','DJ_SMF01_5M','GJ_UPF01_5M','DG_UPF01_5M'], ## 1차 학습
      "SET_2": ['BS_AMF21_5M', 'DJ_UPF01_5M'], # 2차 학습
      "SET_3": ['DJ_SMF03_5M'], #3차 학습
      "SET_4": ['GR_AMF21_5M', 'GR_AMF22_5M', 'GR_SMF01_5M', 'GR_SMF04_5M','GR_UPF01_5M','GR_UPF03_5M','GR_UPF05_5M',
                'GR_UPF06_5M', 'HH_AMF21_5M','HH_AMF22_5M','HH_SMF01_5M','HH_SMF04_5M', 'HH_UPF01_5M','HH_UPF03_5M',
                'HH_UPF04_5M','HH_UPF05_5M', 'HH_UPF06_5M'] # 4차 학습
     }

train_list = []

for idx in trainable_csv_tmp1.index:
    if idx[0] in nf["SET_4"]: # ''DJ_SMF03_5M','
        train_list.append(idx)

train(args,train_list)


###--------------------------------------------------------------------------------------------------------------------     


ModuleNotFoundError: No module named 'preprossesor'

In [5]:
train_list


MultiIndex([('GR_AMF21_5M', 'COSVC|UDMUECM', '/PEER10.124.56.194', ...),
            ('GR_AMF21_5M', 'COSVC|UDMUECM', '/PEER10.124.56.195', ...),
            ('GR_AMF21_5M', 'COSVC|UDMUECM',  '/PEER10.124.56.66', ...),
            ('GR_AMF21_5M', 'COSVC|UDMUECM',  '/PEER10.124.56.67', ...),
            ('GR_AMF21_5M', 'COSVC|UDMUECM', '/PEER10.124.61.194', ...),
            ('GR_AMF21_5M', 'COSVC|UDMUECM', '/PEER10.124.61.195', ...),
            ('GR_AMF21_5M', 'COSVC|UDMUECM',  '/PEER10.124.61.66', ...),
            ('GR_AMF21_5M', 'COSVC|UDMUECM',  '/PEER10.124.61.67', ...),
            ('GR_AMF21_5M', 'COSVC|UDMUECM',  '/PEER10.124.57.66', ...),
            ('GR_AMF21_5M', 'COSVC|UDMUECM',  '/PEER10.124.57.67', ...),
            ...
            ('GR_AMF21_5M',     'PDUSMF|SE',      '/IP39.18.73.3', ...),
            ('GR_AMF21_5M',     'PDUSMF|SE',      '/IP39.18.73.1', ...),
            ('GR_AMF21_5M',     'COSVC|EIR',    '/PEER10.6.238.4', ...),
            ('GR_AMF21_5M',     'CO

In [ ]:
train(args,train_list)

last inference datetime : 2024-04-30 18:00:00 <class 'datetime.datetime'>
train priod : 2024-03-16 18:00:00 2024-04-30 18:00:00
SELECT DATETIME,PORT,SYSTEM, `REG_ATTEMPT(count)` FROM `COSVC|UDMUECM` where DATETIME BETWEEN '2024-03-16 18:00:00' and '2024-04-30 18:00:00'
Data Null States: [0. 0. 0. 0.]
Port List =[20]
/mnt/lv02/ /mnt/lv02/save_path/
                 DATETIME                PORT         SYSTEM  \
0     2024-03-16 18:00:00  /PEER10.124.56.194  GURO_AMF21_SS   
1     2024-03-16 18:10:00  /PEER10.124.56.194  GURO_AMF21_SS   
2     2024-03-16 18:15:00  /PEER10.124.56.194  GURO_AMF21_SS   
3     2024-03-16 18:20:00  /PEER10.124.56.194  GURO_AMF21_SS   
4     2024-03-16 18:25:00  /PEER10.124.56.194  GURO_AMF21_SS   
...                   ...                 ...            ...   
11872 2024-04-30 17:40:00  /PEER10.124.56.194  GURO_AMF21_SS   
11873 2024-04-30 17:45:00  /PEER10.124.56.194  GURO_AMF21_SS   
11874 2024-04-30 17:50:00  /PEER10.124.56.194  GURO_AMF21_SS   
11875 2024

In [ ]:
model_th